# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [ ]:
# Import MongoClient and pretty print
from pymongo import MongoClient
from pprint import pprint

In [ ]:
#Create an instance of the Mongo Client
mongo = MongoClient(port=27017)

In [ ]:
#Display list of databases in Mongo to confirm that the uk_food database is created
print(mongo.list_database_names())

In [ ]:
#Assign the uk_food database to db variable
db = mongo['uk_food']
db

In [ ]:
#Look at the collections in the uk_food database
print(db.list_collection_names())

In [ ]:
#Look at the collections in the uk_food database
print(db.list_collection_names())

In [ ]:
#Display one document in the establishments collection
pprint(db.establishments.find_one())

In [ ]:
# assign the establishments collection to a variable named establishments
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
#Make a dictionary for the new restaurant
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
#Insert new restaurant into the establishments collection
establishments.insert_one(new_restaurant)

In [ ]:
#Confirm that the new restaurant was added
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
#See the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1}
pprint(establishments.find_one(query, projection=fields))

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the BusinessTypeID from above cell
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": 1 }}
)

In [ ]:
#Check that the new restaurant was updated
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
#Find the number of documents that have LocalAuthorityName as "Dover"
dover_documents = establishments.count_documents({"LocalAuthorityName": "Dover"})
dover_documents

In [ ]:
#Delete all documents where LocalAuthorityName is equal to "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

In [ ]:
#Confirm that the dover documents have been deleted
dover_documents = establishments.count_documents({"LocalAuthorityName": "Dover"})
dover_documents

In [ ]:
#Confirm that other documents show with 'find_one'
pprint(establishments.find_one())

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
#Change longitude and latitude data types from String to Double
establishments.update_many({}, [{"$set": {"geocode.longitude": 
                                          {"$toDouble": "$geocode.longitude"},
                                          "geocode.latitude": {
                                              "$toDouble": "$geocode.latitude"
                                          }
                                         }}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change RatingValue data type from String to Integer
establishments.update_many({}, [{"$set": {"RatingValue": 
                                          {"$toInt": "$RatingValue"}}}])

In [ ]:
#Confirm that the coordinates and rating value are now numbers
pprint(establishments.find_one())